In [1]:
%%time 
from data import *
from data.pair_feature import *

CPU times: user 21.7 s, sys: 2.61 s, total: 24.3 s
Wall time: 24.3 s


In [4]:
simple_features_train.shape

(2991396, 9)

In [5]:
simple_features_test.shape

(1044196, 9)

In [6]:
from xgboost.sklearn import XGBClassifier

In [7]:
model = XGBClassifier()

In [14]:
model.fit(simple_features_train, item_pairs_train.isDuplicate, eval_metric='roc_auc')

XGBoostError: b'[10:20:01] src/metric/metric.cc:21: Unknown metric function roc_auc'

In [13]:
simple_features_train.head()

,attrsJSON_item_jaccard,attrsJSON_key_jaccard,price_diff_ratio,same_categoryID,same_description,same_lat_lon,same_locationID,same_metroID,same_title
0,1.0,1.0,0.000000,True,True,True,True,False,True
1,1.0,1.0,0.047619,True,False,True,True,False,False
2,1.0,1.0,0.000000,True,False,True,True,False,False
3,1.0,1.0,0.000000,True,False,False,True,False,False
4,1.0,1.0,0.090909,True,False,True,True,False,False


In [12]:
model.feature_importances_

array([ 0.34999999,  0.09428571,  0.29857144,  0.        ,  0.04571429,
        0.04285714,  0.03571429,  0.05714286,  0.07571428], dtype=float32)